In [1]:
# Import our dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import StratifiedKFold

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
columns = ["hotel","is_canceled","lead_time","arrival_date_year","arrival_date_month",
           "arrival_date_week_number","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights",
           "adults","children","babies","meal","country","market_segment","distribution_channel","is_repeated_guest",
           "previous_cancellations","previous_bookings_not_canceled","reserved_room_type","assigned_room_type","booking_changes",
           "deposit_type","agent","company","days_in_waiting_list","customer_type","adr","required_car_parking_spaces",
           "total_of_special_requests","reservation_status","reservation_status_date"]

target = ["is_canceled"]

In [3]:
hotel_df=pd.read_csv("../Resources/hotel_reservations.csv")
hotel_df = hotel_df.loc[:, columns].copy()
# Drop the null columns where all values are null
hotel_df = hotel_df.dropna(axis='columns', how='all')

# Drop the null rows
hotel_df = hotel_df.dropna()

In [4]:
X = pd.get_dummies(hotel_df, columns=["hotel","country","market_segment","distribution_channel",
           "previous_cancellations","reservation_status","lead_time","arrival_date_year","arrival_date_month",
           "arrival_date_week_number","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights",
           "adults","children","babies","meal", "assigned_room_type","agent", "total_of_special_requests","reservation_status_date","company",
                                        "days_in_waiting_list","customer_type","required_car_parking_spaces","reserved_room_type",
                                                                 "is_repeated_guest","booking_changes",
           "deposit_type","adr","previous_bookings_not_canceled"]).drop(["is_canceled"], axis=1)
# X = pd.get_dummies(hotel_df, columns=['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date']).drop(['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests'], axis=1)

y = hotel_df["is_canceled"]

X.head()

,hotel_City Hotel,hotel_Resort Hotel,country_AUS,country_AUT,country_BEL,country_BGR,country_CHE,country_CHN,country_CZE,country_DEU,...,adr_195.0,adr_212.6,adr_224.0,adr_246.0,previous_bookings_not_canceled_0,previous_bookings_not_canceled_1,previous_bookings_not_canceled_2,previous_bookings_not_canceled_4,previous_bookings_not_canceled_5,previous_bookings_not_canceled_7
2392,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2697,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2867,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2877,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2878,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
# describe () function returns the statistical summary of the DataFrame
X.describe()

,hotel_City Hotel,hotel_Resort Hotel,country_AUS,country_AUT,country_BEL,country_BGR,country_CHE,country_CHN,country_CZE,country_DEU,...,adr_195.0,adr_212.6,adr_224.0,adr_246.0,previous_bookings_not_canceled_0,previous_bookings_not_canceled_1,previous_bookings_not_canceled_2,previous_bookings_not_canceled_4,previous_bookings_not_canceled_5,previous_bookings_not_canceled_7
count,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,...,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000
mean,0.216590,0.783410,0.004608,0.138249,0.013825,0.004608,0.004608,0.009217,0.018433,0.078341,...,0.004608,0.004608,0.004608,0.004608,0.935484,0.027650,0.023041,0.004608,0.004608,0.004608
std,0.412873,0.412873,0.067884,0.345959,0.117034,0.067884,0.067884,0.095781,0.134823,0.269329,...,0.067884,0.067884,0.067884,0.067884,0.246238,0.164346,0.150382,0.067884,0.067884,0.067884
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
y.value_counts()

0    200
1     17
Name: is_canceled, dtype: int64

In [44]:
# splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, test_size=0.3)

In [45]:
# X_train.shape


In [46]:
# Instantiate the model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [47]:
# Applying Recursive Feature Elimination 
import statsmodels.api as sm
from sklearn.feature_selection import RFE
rfe = RFE(classifier, 20)
rfe = rfe.fit(X, y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

C:\Users\krishna\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=20 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


[False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False  True False False  True  True  True False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [48]:
# train the model
classifier.fit(X_train, y_train)

LogisticRegression()

In [49]:
# predict the model
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
18194,0,0
2967,0,0
2969,0,0
3067,1,1
18228,0,0
...,...,...
32385,0,0
18285,0,0
18264,0,0
16645,0,0


In [50]:
# Measure Model Performance
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9393939393939394

In [14]:

# logit_model=sm.Logit(y,X)
# result=logit_model.fit()
# y_pred = classifier.predict(X_test)
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [1]:
# plt.scatter(X_test, y_test, marker="o", c="r")
# X_test["hotel_City Hotel"].describe
# X_test.nunique
# pd.unique(X_test["country_AUS"])
# plt.bar(X_test["country_AUS"], y_test)
# plt.plot(X_test["country_AUS"], y_test, marker="o", c="r")